In [20]:
import numpy as np

from elasticsearch import Elasticsearch
import datetime
import pandas as pd

import os, glob, json

es = Elasticsearch('http://localhost:9200')

In [31]:
def ctime(t):
    return datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S.%f').strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]

starttime_field="1971-12-01 00:00:00.000"
endtime_field="2071-12-01 00:00:00.000"

lon_klick = 116.99127
lat_klick = -1.0615256
varname_field = "sensor0"

query = {
    "bool":{
        "must" : [
            {
                "range": {  # expect this to return the one result on 2012-12-20
                    "starttime1": {
                        "gte": ctime(starttime_field),
                        "lte": ctime(endtime_field)
                    }
                }
            },
            {"exists" : {"field": varname_field}}
        ],
        "filter": {
            "geo_shape": {
                "spatiotempcovera0": {
                    "shape": {
                        "type": "point",
                        "coordinates": [float(lon_klick),
                                        float(lat_klick)]
                    },
                    "relation": "INTERSECTS"
                }
            }
        }    
    }
}

all_data = []
resp = es.search(index='dlrmetadata', doc_type='doc', body={"query":query}, size=1000,   scroll = '2m')
sid = resp['_scroll_id']
scroll_size = resp['hits']['total']
while (scroll_size > 0):
    print(sid, scroll_size)
    all_data = all_data + [d['_source'] for d in resp['hits']['hits']]
    resp = es.scroll(scroll_id = sid, scroll = '2m')
    # Update the scroll ID
    sid = resp['_scroll_id']
    # Get the number of results that we returned in the last scroll
    scroll_size = len(resp['hits']['hits'])

DXF1ZXJ5QW5kRmV0Y2gBAAAAAAAAAwgWdUc0MWxEOXZTY1NvNndJb0s3a1BJQQ== 38


'\nq = {\n        "range" : {\n            "starttime0" : {\n                "gte" : "1970-01-01 15:15:15.151",\n                "lte" : "1970-01-01 15:15:15.152"\n            }\n        }\n    }\n    \n\n\nq = {\n         "match" : { "unique_id" : "dims_op_pl_eoweb1_XXXXB00000000295446012384" }\n    } \n    \nq1 = {\n        "range": {  # expect this to return the one result on 2012-12-20\n            "starttime1": {\n                "gte": ctime(starttime_field),\n                "lte":ctime(endtime_field)\n            }\n        }\n    }\n\n'